In [ ]:
import pandas as pd
import pysubgroup as ps

In [ ]:
data_path = './CC GENERAL.csv'

df = pd.read_csv(data_path)
print("Shape inicial:", df.shape)

Shape inicial: (8950, 18)


In [ ]:
# Como tempos poucos dados, em vez de excluir os valores nulos, vamos substituí-los pela mediana da coluna
df = df.fillna(df.median(numeric_only=True))

## Primeiro Experimento: Identificação de clientes mais endividados

In [13]:
target = ps.NumericTarget('BALANCE')

search_space = ps.create_selectors(df, ignore=['CUST_ID', 'BALANCE'])

# Usaremos a StandardQF, que mede a diferença da média do subgrupo em desvios padrão
# O parâmetro 'a' ajuda a ponderar o tamanho do subgrupo.
quality_function = ps.StandardQFNumeric(a=0.5)

In [ ]:
# Criando a tarefa de descoberta de subgrupos
task = ps.SubgroupDiscoveryTask(
    df,
    target,
    search_space,
    result_set_size=10,  # Queremos os 10 melhores subgrupos
    depth=3,             # Profundidade máxima da descrição
    qf=quality_function
)

# Executando a busca
result = ps.BeamSearch().execute(task)

In [17]:
media_global = df['BALANCE'].mean()
df_results = result.to_dataframe()

print("="*60)
print("Análise Detalhada dos Subgrupos de Maior Impacto")
print("="*60)
print(f"Média Global: R$ {media_global:,.2f}")

for index, row in df_results.iterrows():
    # Coletando os valores da linha
    qualidade = row['quality']
    descricao = row['subgroup']
    tamanho_subgrupo = row['size_sg']
    media_sg = row['mean_sg']

    # Imprimindo o "card" de resultado para cada subgrupo
    print(f"Qualidade: {qualidade:.3f} | Subgrupo: {descricao}")
    print(f"   > Tamanho: {tamanho_subgrupo:,.0f} cartões")
    print(f"   > Média do Subgrupo: R$ {media_sg:,.2f}")
    print("-" * 40)

Análise Detalhada dos Subgrupos de Maior Impacto
Média Global: R$ 1,564.47
Qualidade: 139102.674 | Subgrupo: CREDIT_LIMIT>=7000.0 AND MINIMUM_PAYMENTS>=994.39
   > Tamanho: 803 cartões
   > Média do Subgrupo: R$ 6,473.30
----------------------------------------
Qualidade: 138515.067 | Subgrupo: BALANCE_FREQUENCY>=1.0 AND CREDIT_LIMIT>=7000.0 AND MINIMUM_PAYMENTS>=994.39
   > Tamanho: 740 cartões
   > Média do Subgrupo: R$ 6,656.39
----------------------------------------
Qualidade: 135067.097 | Subgrupo: CREDIT_LIMIT>=7000.0 AND MINIMUM_PAYMENTS>=994.39 AND PRC_FULL_PAYMENT: [0.0:0.08[
   > Tamanho: 708 cartões
   > Média do Subgrupo: R$ 6,640.61
----------------------------------------
Qualidade: 133420.287 | Subgrupo: CREDIT_LIMIT>=7000.0 AND MINIMUM_PAYMENTS>=994.39 AND TENURE>=12
   > Tamanho: 726 cartões
   > Média do Subgrupo: R$ 6,516.17
----------------------------------------
Qualidade: 122549.928 | Subgrupo: MINIMUM_PAYMENTS>=994.39
   > Tamanho: 1,790 cartões
   > Média do S

### Interpretação inicial do primeiro experimento

ATENÇÃO: ANÁLISE FEITA PELO GEMINI

O atributo "BALANCE" significa saldo devedor. Ou seja, ele é o valor total que ainda precisa ser pago na fatura. As características que mais se repetem e definem os grupos com saldo devedor mais alto são:

- **Pagamento Mínimo Elevado:** A condição MINIMUM_PAYMENTS >= 994.39 aparece em quase todas as regras de alta qualidade. Isso é um indicador direto de que o saldo devedor (BALANCE) total é muito alto, já que o pagamento mínimo é um percentual desse saldo. Este é o principal descritor encontrado, mas não descreve padrões muito interessantes, já que as duas variáveis sao correlacionadas.

- **Limite de Crédito Alto:** A segunda condição mais forte é CREDIT_LIMIT >= 7000.0. Clientes com alto limite de crédito tendem a acumular um saldo devedor maior.

- **Não Pagamento do Valor Total:** A regra PRC_FULL_PAYMENT: [0.0:0.08[ (ou seja, o cliente paga entre 0% e 8% do valor total da fatura) é um fator chave. Isso caracteriza o "cliente rotativo", que financia o saldo de um mês para o outro, acumulando juros e, consequentemente, aumentando o BALANCE.

- **Frequência de Atualização do Saldo:** A condição BALANCE_FREQUENCY >= 1.0 indica que o saldo desses clientes é atualizado com muita frequência, sugerindo que são usuários ativos do cartão.

**Insights principais**:
1. **O Saldo Mais Elevado:** O subgrupo com a maior média de saldo devedor (R$ 6.715,71) é aquele que combina limite alto, pagamento mínimo alto e, crucialmente, saques em dinheiro (Cash Advance) elevados (CASH_ADVANCE >= 1.576,49). Isso sugere que o comportamento de saque é um forte acelerador do saldo devedor para clientes que já possuem um perfil de alto endividamento.

2. **Clientes Antigos e Ativos:** Clientes mais antigos (TENURE >= 12) e com saldo frequentemente atualizado (BALANCE_FREQUENCY >= 1.0)  também são predominantes nesses grupos de alto saldo, indicando que esse comportamento de endividamento se consolida com o tempo de uso do cartão.

Em resumo, essa análise identificou com sucesso o perfil do cliente "endividado de alto valor": um cliente antigo, com alto limite, que não quita a fatura e usa o crédito rotativo, muitas vezes potencializado pelo uso de saques (cash advance).